In [1]:
import pandas as pd
import numpy as np
import scipy
from pandas import read_json, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [4]:
file = 'kenya-climate-data-1991-2016-temp-degress-celcius-Copy1.csv'

In [5]:
def read_datas(file_obj, format=None):
        # the current format is a list of dictionaries
#         file_obj = request.data

        # read list of dict into dict, if json, use read_json
#         weather = DataFrame(file_obj)
        weather = pd.read_csv(file_obj)

        # clean up the data
        weather = weather.rename(columns={'Month Average': 'months', 'Temperature - (Celsius)': 'temp', 'Rainfall - (MM)': 'rain'})
        weather['months'] = weather['months'].str.replace(' Average', '')       
        d = dict(zip(pd.date_range('2000-01-01', freq='M', periods=12).strftime('%b'), range(1,13)))
        weather['months_num'] = weather.months.map(d)
        weather.drop('months', axis=1, inplace=True)
        weather['months'] = weather['Year'].astype('str') + '-' + weather['months_num'].astype('str') + '-01'
        weather['months'] = pd.to_datetime(weather.months)        
        weather.drop(['Year', 'months_num'], axis=1, inplace=True)
        weather.set_index('months', inplace=True)

        rnn_train = weather[:300]
        rnn_test = weather[300:]

        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler()
        # print("This is rnn_train", rnn_train)
        scaler.fit(rnn_train)
        scaled_train = scaler.transform(rnn_train)
        scaled_test = scaler.transform(rnn_test)

        # from tensorflow import keras
        from keras.preprocessing.sequence import TimeseriesGenerator

        # define generator
        n_input = 12
        n_features = 2
        generator = TimeseriesGenerator(
            scaled_train, scaled_train, length=n_input, batch_size=1)

        from keras.models import Sequential
        from keras.layers import Dense
        from keras.layers import LSTM

        # define model
        model = Sequential()
        model.add(LSTM(100, activation='relu',
                  input_shape=(n_input, n_features)))
        model.add(Dense(2))
        model.compile(optimizer='adam', loss='mse')

        model.summary()
        model.fit(generator, epochs=50)

        # last_train_batch = scaled_train[-12:]
        # last_train_batch = last_train_batch.reshape((1, n_input, n_features))
        # model.predict(last_train_batch)

        test_predictions = []

        first_eval_batch = scaled_train[-n_input:]
        current_batch = first_eval_batch.reshape((1, n_input, n_features))

        for i in range(len(rnn_test)):
            # get the prediction value for the first batch
            current_pred = model.predict(current_batch)[0]

            # append the prediction into the array
            test_predictions.append(current_pred)

            # use the prediction to update the batch and remove the first value
            current_batch = np.append(current_batch[:, 1:, :], [
                                      [current_pred]], axis=1)

        true_predictions = scaler.inverse_transform(test_predictions)
        rnn_test[['temp_preds', 'rain_preds']] = true_predictions

        # print("This is rnn test", rnn_test.to_json())
        return rnn_test.to_json()

#         return Response({
#             "data": rnn_test.to_json(),})
        # return Response({"data": 20})

In [6]:
read_datas(file)

ImportError: dlopen(/Users/stevedavieswambui/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/_pywrap_tfe.so, 0x0002): Library not loaded: @rpath/_pywrap_tensorflow_internal.so
  Referenced from: <F4DA3910-136E-3F7E-9843-663DB4F505C6> /Users/stevedavieswambui/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/_pywrap_tfe.so
  Reason: tried: '/Users/stevedavieswambui/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utfe.so___Utensorflow/_pywrap_tensorflow_internal.so' (no such file), '/Users/stevedavieswambui/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/../../_solib_darwin_x86_64/_U_S_Stensorflow_Spython_C_Upywrap_Utensorflow_Uinternal_Umacos___Utensorflow_Spython/_pywrap_tensorflow_internal.so' (no such file), '/Users/stevedavieswambui/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so' (no such file), '/Users/stevedavieswambui/opt/anaconda